In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from matplotlib import font_manager, rc
import numpy as np
import csv
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz

In [2]:
# 한글 폰트 적용 - 한글깨짐 방지
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# csv 파일 읽어와서 데이터프레임에 저장
df_source = pd.read_csv('NHIS_OPEN_GJ_2017_3.csv', encoding='cp949')
df_source = df_source.drop_duplicates() # 중복값 제거
df_source = df_source.dropna() # 결측값 제거
df_source

,Unnamed: 0,번호,성별,연령,신장,체중,허리둘레,수축기혈압,이완기혈압,식전혈당,총콜레스테롤,흡연상태,음주여부
0,0,1,1,40,170,75,90.0,120.0,80.0,99.0,193.0,1.0,1.0
1,1,2,1,35,180,80,89.0,130.0,82.0,106.0,228.0,3.0,0.0
2,2,3,1,45,165,75,91.0,120.0,70.0,98.0,136.0,1.0,0.0
3,3,4,1,55,175,80,91.0,145.0,87.0,95.0,201.0,1.0,0.0
4,4,5,1,55,165,60,80.0,138.0,82.0,101.0,199.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199772,199772,199996,1,45,170,100,107.0,135.0,88.0,112.0,247.0,3.0,1.0
199773,199773,199997,2,30,155,45,63.0,107.0,61.0,83.0,151.0,1.0,1.0
199774,199774,199998,1,55,160,70,91.0,100.0,76.0,100.0,222.0,1.0,0.0
199775,199775,199999,1,40,170,75,88.2,147.0,89.0,81.0,125.0,3.0,1.0


In [4]:
x = df_source.drop(['성별'], axis=1) # 성별특성을 제외한 모든 특성의 데이터
y = df_source['성별']

In [5]:
# 훈련, 테스트 셋 분리
# test_size=0.3으로 설정하여 훈련:테스트 비율을 7:3으로
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

treg = tree.DecisionTreeClassifier() # 결정트리 모델 생성
treg = treg.fit(x_train, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.8925
훈련 셋 정확도: 1.0000


In [6]:
# 테스트 셋 에서는 89%, 훈련 셋 에서는 100%의 정확도가 나타난다

In [7]:
# 각 특성별 확률값(중요도) 출력
feature_name_list = x_train.columns
df_feature_importance = treg.feature_importances_
for i,v in enumerate(df_feature_importance):
    print('(%2d) Feature : %s, %5f' %(i, feature_name_list[i], v))

( 0) Feature : Unnamed: 0, 0.023366
( 1) Feature : 번호, 0.022103
( 2) Feature : 연령, 0.053951
( 3) Feature : 신장, 0.549321
( 4) Feature : 체중, 0.014440
( 5) Feature : 허리둘레, 0.069385
( 6) Feature : 수축기혈압, 0.024508
( 7) Feature : 이완기혈압, 0.024575
( 8) Feature : 식전혈당, 0.031472
( 9) Feature : 총콜레스테롤, 0.036959
(10) Feature : 흡연상태, 0.142331
(11) Feature : 음주여부, 0.007588


In [8]:
#각 특성을 확률값이 큰 순서로 정렬
df_importance = pd.DataFrame({'feature' : x_train.columns, 'importance' : treg.feature_importances_})
df_importance = df_importance.sort_values(by='importance', ascending=False)
df_importance.head(10)

,feature,importance
3,신장,0.549321
10,흡연상태,0.142331
5,허리둘레,0.069385
2,연령,0.053951
9,총콜레스테롤,0.036959
8,식전혈당,0.031472
7,이완기혈압,0.024575
6,수축기혈압,0.024508
0,Unnamed: 0,0.023366
1,번호,0.022103


In [9]:
# 신장, 흡연상태, 허리둘레, 연령이 높은 확률값을 나타낸다

In [10]:
del x, y, x_train, x_test, y_train, y_test, treg #메모리 반환, 초기화

In [11]:
#이번에는 연령, 번호 특성도 제외하고 특성 수를 줄여서 알고리즘을 실행해본다

x = df_source.drop(['성별', '연령', '번호'], axis=1) # 3개의 특성을 제외한 모든 특성의 데이터
y = df_source['성별']

In [12]:
# 훈련, 테스트 셋 분리
# test_size=0.3으로 설정하여 훈련:테스트 비율을 7:3으로
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

treg = tree.DecisionTreeClassifier() # 결정트리 모델 생성
treg = treg.fit(x_train, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.8782
훈련 셋 정확도: 1.0000


In [13]:
# 이전 보다 테스트 셋 정확도가 2% 감소하였다

In [14]:
del x, y, x_train, x_test, y_train, y_test, treg #메모리 반환, 초기화

In [15]:
#이번에는 StandardScaler()을 이용화 데이터를 표준화 한 후에 알고리즘을 실행해본다

x = df_source.drop(['성별', '번호'], axis=1) # 2개의 특성을 제외한 모든 특성의 데이터
y = df_source['성별']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# 표준화 전처리 적용
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_train_scaler = pd.DataFrame(x_train_scaler, columns = x_train.columns) # 표준화를 적용한 이후 다시 컬럼명을 지정해야 한다 

del scaler
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaler = scaler.transform(x_test)
x_test_scaler = pd.DataFrame(x_test_scaler, columns = x_test.columns)

In [17]:
treg = tree.DecisionTreeClassifier() # 결정트리 모델 생성
treg = treg.fit(x_train_scaler, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test_scaler, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train_scaler, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.8908
훈련 셋 정확도: 1.0000


In [18]:
# 데이터를 표준화한 결과 이전과 크게 다르지 않은 결과가 나타났다.

In [19]:
# 각 특성별 확률값(중요도) 출력
feature_name_list = x_train.columns
df_feature_importance = treg.feature_importances_
for i,v in enumerate(df_feature_importance):
    print('(%2d) Feature : %s, %5f' %(i, feature_name_list[i], v))

( 0) Feature : Unnamed: 0, 0.040656
( 1) Feature : 연령, 0.054206
( 2) Feature : 신장, 0.549375
( 3) Feature : 체중, 0.015148
( 4) Feature : 허리둘레, 0.069700
( 5) Feature : 수축기혈압, 0.025488
( 6) Feature : 이완기혈압, 0.025593
( 7) Feature : 식전혈당, 0.032082
( 8) Feature : 총콜레스테롤, 0.037809
( 9) Feature : 흡연상태, 0.142294
(10) Feature : 음주여부, 0.007650


In [20]:
#각 특성을 확률값이 큰 순서로 정렬
df_importance = pd.DataFrame({'feature' : x_train.columns, 'importance' : treg.feature_importances_})
df_importance = df_importance.sort_values(by='importance', ascending=False)
df_importance.head(10)

,feature,importance
2,신장,0.549375
9,흡연상태,0.142294
4,허리둘레,0.069700
1,연령,0.054206
0,Unnamed: 0,0.040656
8,총콜레스테롤,0.037809
7,식전혈당,0.032082
6,이완기혈압,0.025593
5,수축기혈압,0.025488
3,체중,0.015148


In [21]:
del x, y, x_train, x_test, y_train, y_test, treg #메모리 반환, 초기화

In [22]:
#이번에는 결정트리 모델에서 max_depth 매개변수를 설정하여 결과를 비교하자

x = df_source.drop(['성별', '번호'], axis=1) # 2개의 특성을 제외한 모든 특성의 데이터
y = df_source['성별']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# 표준화 전처리 적용
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_train_scaler = pd.DataFrame(x_train_scaler, columns = x_train.columns) # 표준화를 적용한 이후 다시 컬럼명을 지정해야 한다 

del scaler
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaler = scaler.transform(x_test)
x_test_scaler = pd.DataFrame(x_test_scaler, columns = x_test.columns)

In [24]:
# max_depth를 10으로 설정
treg = tree.DecisionTreeClassifier(max_depth=10) # 결정트리 모델 생성
treg = treg.fit(x_train_scaler, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test_scaler, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train_scaler, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.9265
훈련 셋 정확도: 0.9366


In [25]:
#이전과 비교했을 때 테스트 셋 정확도는 증가했고, 훈련 셋 정확도는 감소했다

In [36]:
del x, y, x_train, x_test, y_train, y_test, treg #메모리 반환, 초기화

In [37]:
x = df_source.drop(['성별', '번호'], axis=1) # 2개의 특성을 제외한 모든 특성의 데이터
y = df_source['성별']

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# 표준화 전처리 적용
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_train_scaler = pd.DataFrame(x_train_scaler, columns = x_train.columns) # 표준화를 적용한 이후 다시 컬럼명을 지정해야 한다 

del scaler
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaler = scaler.transform(x_test)
x_test_scaler = pd.DataFrame(x_test_scaler, columns = x_test.columns)

In [39]:
# max_depth를 30으로 설정
treg = tree.DecisionTreeClassifier(max_depth=30) # 결정트리 모델 생성
treg = treg.fit(x_train_scaler, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test_scaler, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train_scaler, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.8913
훈련 셋 정확도: 0.9996


In [40]:
# max_depth를 30으로 증가시켰더니 테스트 셋 정확도가 감소하고, 훈련 셋 정확도가 감소했다

In [41]:
del x, y, x_train, x_test, y_train, y_test, treg #메모리 반환, 초기화

In [42]:
x = df_source.drop(['성별', '번호'], axis=1) # 2개의 특성을 제외한 모든 특성의 데이터
y = df_source['성별']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# 표준화 전처리 적용
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_train_scaler = pd.DataFrame(x_train_scaler, columns = x_train.columns) # 표준화를 적용한 이후 다시 컬럼명을 지정해야 한다 

del scaler
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaler = scaler.transform(x_test)
x_test_scaler = pd.DataFrame(x_test_scaler, columns = x_test.columns)

In [43]:
# max_depth를 5으로 설정
treg = tree.DecisionTreeClassifier(max_depth=10) # 결정트리 모델 생성
treg = treg.fit(x_train_scaler, y_train)

# 테스트 셋 정확도 
accuracy = float(treg.score(x_test_scaler, y_test))
print('테스트 셋 정확도: %.4f' %accuracy)

# 훈련 셋 정확도 
accuracy = float(treg.score(x_train_scaler, y_train))
print('훈련 셋 정확도: %.4f' %accuracy)

테스트 셋 정확도: 0.9264
훈련 셋 정확도: 0.9366
